In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':'1ZRjbZ2kdmz8W2mxjke5O9aqsjMKSLtXR'})
downloaded.GetContentFile('training_data.csv') 
downloaded2 = drive.CreateFile({'id':'1NHrpk4osLxwwYEyDgcRDYy1paWZd2xR3'})
downloaded2.GetContentFile('test_data.csv')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import statistics
train_data = pd.read_csv("training_data.csv")
test_data = pd.read_csv("test_data.csv")
ID = test_data.Loan_ID
target = train_data.Loan_Status

from sklearn.preprocessing import LabelEncoder
leb = LabelEncoder()
target = leb.fit_transform(target)
train_d = train_data.drop("Loan_Status",axis=1)

In [0]:
print test_data.shape

(367, 12)


In [0]:
combined_data = pd.concat([train_d,test_data],axis=0)
combined_data.drop("Loan_ID",axis=1,inplace=True)
combined_data["Gender"] = combined_data["Gender"].fillna("Male")
combined_data["Married"] = combined_data["Married"].fillna("Yes")
combined_data["Dependents"] = combined_data["Dependents"].fillna("0")
combined_data["Self_Employed"] = combined_data["Self_Employed"].fillna("No")
combined_data["LoanAmount"] = combined_data["LoanAmount"].fillna(combined_data["LoanAmount"].median())
combined_data["Loan_Amount_Term"] = combined_data["Loan_Amount_Term"].fillna(combined_data["Loan_Amount_Term"].median())
combined_data["Credit_History"] = combined_data["Credit_History"].fillna(combined_data["Credit_History"].max())

from sklearn.preprocessing import LabelEncoder
leb = LabelEncoder()
combined_data["Gender"] = leb.fit_transform(combined_data["Gender"])
combined_data["Married"] = leb.fit_transform(combined_data["Married"])
combined_data["Self_Employed"] = leb.fit_transform(combined_data["Self_Employed"])
combined_data["Property_Area"] = leb.fit_transform(combined_data["Property_Area"])
combined_data["Education"] = leb.fit_transform(combined_data["Education"])


In [0]:
depnd = []
for i in range(0,len(combined_data["Dependents"])):
	if combined_data["Dependents"].iloc[i] == "0":
		depnd.append(0)
	elif combined_data["Dependents"].iloc[i] == "1":
		depnd.append(1)
	elif combined_data["Dependents"].iloc[i] == "2":
		depnd.append(2)
	elif combined_data["Dependents"].iloc[i] == "3+":
		depnd.append(3)

    
combined_data["depnd"] = depnd

In [0]:
combined_data.drop("Dependents",axis=1,inplace=True)
INCOME = 0.8*combined_data["ApplicantIncome"] + 0.2*combined_data["CoapplicantIncome"]
combined_data["INCOME"] = INCOME
R = 0.01
EMI = (combined_data["LoanAmount"]*0.01*(1.01)**combined_data["Loan_Amount_Term"])/(1.01**(combined_data["Loan_Amount_Term"]))
combined_data["EMI"] = EMI
WEIGHT = combined_data["LoanAmount"]/((combined_data["ApplicantIncome"]) + combined_data["CoapplicantIncome"])
combined_data["WEIGHT"] = WEIGHT


In [0]:
all_data = combined_data
from sklearn.preprocessing import scale
combined_data = scale(combined_data)

data_train = combined_data[0:614,:]
data_test =  combined_data[614::,:]
act_test = data_test.copy()

In [0]:
data_test.shape

(367, 14)

In [0]:
from imblearn.over_sampling import ADASYN
sm = ADASYN()
X,Y = sm.fit_sample(data_train,target)
from sklearn.utils import shuffle
for i in range(200):
    X,Y = shuffle(X,Y,random_state=0)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,make_scorer
score = make_scorer(f1_score)
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV,cross_val_score,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier

from sklearn.metrics import f1_score,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
score = make_scorer(f1_score)
model1 = RandomForestClassifier(n_estimators=500,max_features=4,max_depth=5)
model2 = ExtraTreesClassifier(max_depth = 5,n_estimators=500,random_state=42,n_jobs=-1)
model3 = AdaBoostClassifier(extree_tree_clf,n_estimators=500,algorithm="SAMME.R",learning_rate=0.05)
model4 = lgb.LGBMClassifier(learning_rate=0.01,objective='binary')

In [0]:
random_grid = {'n_estimators': [100,200,300,400,500],
               'max_features': [3,4,5,6,7],
               'max_depth': [5,8,10,12,15],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': "bootstrap"}
random_rf = RandomizedSearchCV(estimator = model1, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring=score)
random_rf.fit(X,Y)
print random_rf.best_params_


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.0min finished


{'bootstrap': 'o', 'min_samples_leaf': 1, 'n_estimators': 300, 'max_features': 3, 'min_samples_split': 2, 'max_depth': 15}


In [0]:
random_grid = {"gamma":[0.01,0.05,0.1,1,5],
               "C":[0.1, 1, 10, 100, 1000],
               "probability":[True],
               "kernel":['linear', 'rbf']}
random_svm = RandomizedSearchCV(estimator = model2, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1,scoring=score)
random_svm.fit(X,Y)
print random_svm.best_params_


In [0]:
train_model1 = model1.fit(X,Y)
train_model2 = model2.fit(X,Y)
train_model3 = model3.fit(X,Y)
train_model4 = model4.fit(X,Y)

In [0]:
ss = train_model4.predict_proba(data_test)
print ss

[[0.41815372 0.58184628]
 [0.41603528 0.58396472]
 [0.4233774  0.5766226 ]
 [0.39888221 0.60111779]
 [0.53059844 0.46940156]
 [0.48428904 0.51571096]
 [0.4708538  0.5291462 ]
 [0.87211272 0.12788728]
 [0.39676022 0.60323978]
 [0.46985213 0.53014787]
 [0.52228009 0.47771991]
 [0.45878364 0.54121636]
 [0.47541715 0.52458285]
 [0.82019647 0.17980353]
 [0.48543605 0.51456395]
 [0.52142463 0.47857537]
 [0.42190754 0.57809246]
 [0.38937921 0.61062079]
 [0.43613648 0.56386352]
 [0.49116693 0.50883307]
 [0.4219111  0.5780889 ]
 [0.50281698 0.49718302]
 [0.51615328 0.48384672]
 [0.46835915 0.53164085]
 [0.43615414 0.56384586]
 [0.63121053 0.36878947]
 [0.4190366  0.5809634 ]
 [0.43812653 0.56187347]
 [0.38072596 0.61927404]
 [0.5045441  0.4954559 ]
 [0.45976923 0.54023077]
 [0.42174346 0.57825654]
 [0.48729966 0.51270034]
 [0.41731568 0.58268432]
 [0.40452954 0.59547046]
 [0.84031347 0.15968653]
 [0.3951106  0.6048894 ]
 [0.38876332 0.61123668]
 [0.51885908 0.48114092]
 [0.51289175 0.48710825]


In [0]:
actual_data = pd.DataFrame(X,Y) 
actual_data.names =['Gender', 'Married', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'depnd',
       'INCOME', 'EMI', 'WEIGHT','Loan_Status']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


In [0]:
def calculate_strong_prob(X,Y,data_test):
  train_model1 = model1.fit(X,Y)
  train_model2 = model2.fit(X,Y)
  train_model3 = model3.fit(X,Y)
  train_model4 = model4.fit(X,Y)
  list1 = [train_model1,train_model2,train_model3,train_model4]
  test_ex_prob = []
  for i in range(len(data_test)):
    zero_prob = []
    one_prob = []
    for model in list1:
      model_pred = model.predict_proba(data_test[i].reshape(1,14))
      zero_prob.append(model_pred[0][0])
      one_prob.append(model_pred[0][1])
    zero_mean = np.mean(zero_prob)
    one_mean = np.mean(one_prob)
    if one_mean > zero_mean:
      test_ex_prob.append([one_mean,1])
    else:
      test_ex_prob.append([zero_mean,0])
    prob_test = pd.DataFrame(test_ex_prob,columns=["probability","class"]).sort_index()
    sort_prob = prob_test.sort_values('probability',ascending=False)
  return sort_prob



In [0]:

while len(data_test)>=4:
  modified_data = calculate_strong_prob(X,Y,data_test)
  add_to_train = modified_data.iloc[0:4,:]
  index_high_prob = list(add_to_train.index)
  high_prob_test_ex = data_test[index_high_prob,:]
  high_prob_test_label = add_to_train["class"].values
  update_X = np.concatenate([X,high_prob_test_ex]) 
  update_Y = np.concatenate([Y,high_prob_test_label])
  update_test = np.delete(data_test,index_high_prob,0)
  print(update_test.shape)
  X = update_X
  Y = update_Y
  data_test = update_test

In [0]:
clf3 = GaussianNB()
rnd_clf = RandomForestClassifier(max_depth = 5,n_estimators=500,random_state=42,n_jobs=-1)
extree_tree_clf = ExtraTreesClassifier(max_depth = 5,n_estimators=500,random_state=42,n_jobs=-1)
ada_clf = AdaBoostClassifier(extree_tree_clf,n_estimators=500,algorithm="SAMME.R",learning_rate=0.05)
mlp_clf = MLPClassifier(random_state=42)
model_LGBM =lgb.LGBMClassifier(learning_rate=0.01,objective='binary')
from sklearn.ensemble import VotingClassifier
named_estimators = [("rnd_clf",rnd_clf),("extree_tree_clf",extree_tree_clf),("ada_clf",ada_clf)]
voting_clf = VotingClassifier(named_estimators,voting="soft",flatten_transform=True)
voting_clf.fit(X,Y)
test_predict = voting_clf.predict(act_test)
acc = []
for i in range(len(test_predict)):
	if test_predict[i] == 1:
		acc.append("Y")
	else:
		acc.append("N")

test_result = pd.concat([ID,pd.DataFrame(acc)],axis=1)
test_result.columns = ["Loan_ID","Loan_Status"]
test_result.to_csv("loan_prediction202.csv", encoding='utf-8', index=False)
from google.colab import files
files.download("loan_prediction202.csv") 


In [0]:
test_result.to_csv("loan_prediction202.csv", encoding='utf-8', index=False)
from google.colab import files
files.download("loan_prediction202.csv") 

In [0]:
model1.fit(X,Y)
test_predict = model1.predict(act_test)
acc = []
for i in range(len(test_predict)):
	if test_predict[i] == 1:
		acc.append("Y")
	else:
		acc.append("N")

test_result = pd.concat([ID,pd.DataFrame(acc)],axis=1)
test_result.columns = ["Loan_ID","Loan_Status"]
test_result.to_csv("loan_prediction201.csv", encoding='utf-8', index=False)
from google.colab import files
files.download("loan_prediction201.csv") 

